#**Microsoft Identity for All : Hackathon for Microsoft Employees**



### **Nina Carlson**

Explore Intern

IES | HCCS | Service Enablement

image.png

# **FaceBlur**

FaceBlur is a tool that detects faces in images using the Azure Cognitive Services Face API and blurs them to anonymize the information as a security and privacy measure. 

FaceBlur was created in Google Colab (notebook) and is written in Python. The tool leverages the Azure Cognitive Services Face API. 

Find more information about the Azure Face API here: 
https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/overview-identity



In [ ]:
#Imports
import os,sys,json,glob,requests,cv2

import matplotlib.pyplot as plt
from matplotlib import patches
from PIL import Image
from io import BytesIO
import json as js

#Display images within notebook
%matplotlib inline

In [ ]:
#Loads info from config file 
with open('config.json','r') as file:
    config = json.load(file)

In [ ]:
#Instantiate variables and parameters needed to call the Face API 
#Variables
_url = config['face_api_endpoint']
_url_detection_face = _url + config['face_detection_url']
_key = config['face_api_key']
_maxNumRetries = 10

#params
params = {
    'detectionModel':'detection_02',
    'returFaceId': 'true'
}

#headers
headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key

In [ ]:
#Run this cell to load an img from url

#Load image from url
image = '' #insert the url of image
headers['Content-Type'] = 'application/json'
data = None
js = {'url': image}

In [1]:
#Option to Load local file
# headers['Content-Type'] = 'application/octet-stream'
# js = None

# with open('/content/temp/test.png','rb') as f:
#     data = f.read()

In [ ]:
#post image, return dimensions of face in JSON format

response = requests.post(_url_detection_face,params=params,headers=headers, json=js,data=data)
faces_detected = response.json()
print(response.json())

In [ ]:
#Plot detected faces (from url image)
get_img = requests.get('')
img = Image.open(BytesIO(get_img.content))

plt.figure(figsize=(8,8))
ax = plt.imshow(img, alpha=0.6)
for face in faces_detected:
    fr = face["faceRectangle"]
    origin = (fr["left"], fr["top"])
    p = patches.Rectangle(origin, fr["width"], fr["height"], fill=False, linewidth=2, color='b')
    ax.axes.add_patch(p)
_ = plt.axis("off")

In [ ]:
#Helper function to temp download the image in the temp folder
def downloadImg(img,img_name):
    path = "./temp/" + img_name + ".png"
    file = open(path, "wb")
    file.write(img.content)
    file.close()
    print('Img file downloaded')
    return path

In [ ]:
#Helper function to blur all the faces detected and save the file
def blurImg(img,height,width,origin):
    output = "./blurred_img/blurred_img.jpg"
    #Read img
    im = cv2.imread(path)
    #Create ROI
    h,w = height,width
    x,y = origin[0],origin[1]
    roi = im[int(y):int(y)+int(h),int(x):int(x)+int(w)]
    #Blur image in ROI
    blurred_img = cv2.GaussianBlur(roi,(91,91),0)
    #Add blur to the overall img
    im[int(y):int(y)+int(h),int(x):int(x)+int(w)] = blurred_img
    #Save blurred img
    result=cv2.imwrite(r'./blurred_img/blurred_img.jpg', im)
    if result==True:
        print("File saved successfully")
    else:
        print("Error in saving file")
    return output

In [ ]:
#Applies the blur to the faces detected
path = downloadImg(get_img,"test")

for face in faces_detected:
    fr = face["faceRectangle"]
    origin = (fr["left"], fr["top"])
    path = blurImg(path,fr['height'],fr['width'],origin)